# NCAR Earth System Data Science WIP Talk
__This presentation is based on work I did during the [NCAR Summer Internship in Parallel Computational Science (SIParCS) program](https://www2.cisl.ucar.edu/siparcs)__
### Lucas Sterzinger -- Atmospheric Science PhD Candidate at UC Davis
* [Twitter](https://twitter.com/lucassterzinger)
* [GitHub](https://github.com/lsterzinger)
* [Website](https://lucassterzinger.com)



#  Motivation:
* NetCDF is not cloud optimized
* Other formats, like Zarr, aim to 

# What do I mean when I say "Cloud Optimized"?
![Move to cloud diagram](images/cloud-move.png)

In traditional scientific workflows, data is archived in a repository and downloaded to a separate computer for analysis (left). However, datasets are becoming much too large to fit on personal computers, and transferring full datasets from an archive to a seperate machine can use lots of bandwidth.

In a cloud environment, the data can live in object storage (e.g. AWS S3), and analysis can be done in an adjacent compute instances, allowing for low-latency and high-bandwith access to the dataset.

## Why NetCDF doesn't work well in this workflow

NetCDF is probably the most common binary data format for atmospheric/earth sciences, and has a lot of official and community support. However, the NetCDF format/API requires either a) loading the entire dataset in order to access the header/metadata and retreive a chunk of data.

![NetCDF File Object](images/single_file_object.png)

## The Zarr Solution
The [Zarr data format](https://zarr.readthedocs.io/en/stable/) alleviates this problem by storing the metadata and chunks in seperate files that can be accessed as-needed and in parallel.

![Zarr](images/zarr.png)

## _However_
While Zarr proves to be very good for this cloud-centric workflow, most cloud-available data is currently only available in NetCDF/HDF5/GRIB2 format. While it would be _wonderful_ if all this data converted to Zarr overnight, in the meantime it would be great if there was a way to use some of the Zarr spec, right?

# Introducting `fsspec-reference-maker`
[Github page](https://github.com/intake/fsspec-reference-maker)

`fsspec-reference-maker` works by analysing the NetCDF header/metadata info, extracting byte-ranges for each variable chunk, and creating a Zarr-spec metadata file. This file is plaintext and can opened and analyzed with xarray very quickly. When a user requests a certain chunk of data, the NetCDF4 API is bypassed entirely and the Zarr API is used to extract the specified byte-range.

![reference-maker vs zarr](images/referencemaker_v_zarr.png)

***
# Let's try it out!

### Import `fsspec-reference-maker` and make sure it's at the latest version (`0.0.3` at the time of writing)

In [ ]:
import fsspec_reference_maker
fsspec_reference_maker.__version__

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from fsspec_reference_maker.hdf import SingleHdf5ToZarr
from fsspec_reference_maker.combine import MultiZarrToZarr
import fsspec

### Setup an S3 filesystem for listing GOES files on S3

In [ ]:
fs = fsspec.filesystem('s3', anon=True)

In [ ]:
flist = fs.glob("s3://noaa-goes16/ABI-L2-SSTF/2020/210/*/*.nc")
# flist = fs.glob("s3://noaa-goes16/ABI-L2-MCMIPC/2021/049/*/*.nc")


### Prepend `s3://` to the URLS

In [ ]:
flist = ['s3://' + f for f in flist]

### Start a dask cluster

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
import dask.bag as db
flist_bag = db.from_sequence(flist, npartitions=len(flist))
flist_bag

### Definte function to return a reference dictionary for a given S3 file URL

In [ ]:
def gen_ref(f):
    so = dict(
        mode="rb", anon=True, default_fill_cache=False, default_cache_type="none"
    )

    with fsspec.open(f, **so) as infile:
        return SingleHdf5ToZarr(infile, f, inline_threshold=300).translate()

### Map `gen_ref` to each member of `flist_bag` and compute
_Note: if running interactively on Binder, this will take a while since only one worker is available and the references will have to be generated in serial_

In [ ]:
%time dicts = flist_bag.map(gen_ref).compute()

The individual dictionaries can be saved as JSON files if desired

In [21]:
d = dicts[0]

In [23]:
d['templates']['u']

's3://noaa-goes16/ABI-L2-SSTF/2020/210/00/OR_ABI-L2-SSTF-M6_G16_s20202100000205_e20202100059513_c20202100105456.nc'

In [ ]:
import ujson
for d in dicts:
    # Generate name from corresponding URL:
    # Grab URL, strip everything but the filename, 
    # and replace .nc with .json
    name = d['templates']['u'].split('/')[-1].replace('.nc', '.json')

    with open(name, 'w') as outf:
        outf.write(ujson.dumps(d))

### Use `MultiZarrToZarr` to combine the 24 individual references into a single reference
In this example we passed a list of reference dictionaries, but you can also give it a list of `.json` filepaths

In [ ]:
mzz = MultiZarrToZarr(
    dicts,
    remote_protocol='s3',
    remote_options={'anon':True},
    xarray_open_kwargs={
        "decode_cf" : False,
        "mask_and_scale" : False,
        "decode_times" : False,
        "decode_timedelta" : False,
        "use_cftime" : False,
        "decode_coords" : False
    },
    xarray_concat_args={'dim' : 't'}
)

References can be saved to a file (`combined.json`) or passed back as a dictionary (`mzz_dict`)

In [ ]:
mzz.translate('./combined.json')
# mzz_dict = mzz.translate()

***

# Read the referenced files with `fsspec` and `xarray`

In [ ]:
import metpy
import cartopy.crs as ccrs
import hvplot.xarray

Use metpy's `parse_cf()` to generate projection information for future plotting

In [ ]:
fs2 = fsspec.filesystem('reference', fo="./example_jsons/combined.json", remote_protocol='s3', remote_options=dict(anon=True), skip_instance_cache=True)
ds = xr.open_dataset(fs2.get_mapper(""), engine='zarr').metpy.parse_cf()
ds

Use metpy to calculate lat/lon based on the GOES projection grid, and rename time dimension (for better plotting with hvplot)

In [ ]:
ds = ds.metpy.assign_latitude_longitude()
ds['t'].attrs['long_name'] = 'Time'

In [ ]:
mask_lat = (ds.latitude > 30) & (ds.latitude < 60)
mask_lon = (ds.longitude > -90) & (ds.longitude < -60)


sst = ds.SST.where(mask_lat & mask_lon, drop=True)

In [ ]:
sst.hvplot.quadmesh(
    'x', 'y', groupby='t',
    crs = sst.metpy.cartopy_crs, projection=ccrs.Orthographic(-75,30),
    project=True, rasterize=True, coastline=True
)

In [ ]:
sst.mean(dim='t').hvplot.quadmesh(
    'x', 'y',
    crs = sst.metpy.cartopy_crs, projection=ccrs.Orthographic(-75,30),
    project=True, rasterize=True, coastline=True
)